## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
pwd

'C:\\Users\\m163075\\UofM-Boot Camp\\Module 6 - APIs\\Vacation_Search'

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Percent Humidity,Precent Cloudiness,Wind Speed,Current Description
0,0,Baruun-Urt,MN,46.6806,113.2792,-3.87,96,29,2.46,scattered clouds
1,1,Norman Wells,CA,65.2820,-126.8329,-11.16,64,100,21.85,light snow
2,2,Torbay,CA,47.6666,-52.7314,41.58,90,75,23.02,light intensity drizzle
3,3,Azuaga,ES,38.2667,-5.6833,60.46,54,0,0.09,clear sky
4,4,Haines Junction,CA,60.7522,-137.5108,33.75,86,89,3.44,overcast clouds
5,5,Labuhan,ID,-6.8844,112.2051,76.71,89,100,9.84,overcast clouds
6,6,Mar Del Plata,AR,-38.0023,-57.5575,78.82,69,0,11.50,clear sky
7,7,Kapaa,US,22.0752,-159.3190,70.20,84,13,8.79,light rain
8,8,Carnarvon,AU,-24.8667,113.6333,77.07,73,0,16.11,clear sky
9,9,Ushuaia,AR,-54.8000,-68.3000,58.62,35,40,13.80,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Percent Humidity,Precent Cloudiness,Wind Speed,Current Description
5,5,Labuhan,ID,-6.8844,112.2051,76.71,89,100,9.84,overcast clouds
6,6,Mar Del Plata,AR,-38.0023,-57.5575,78.82,69,0,11.50,clear sky
7,7,Kapaa,US,22.0752,-159.3190,70.20,84,13,8.79,light rain
8,8,Carnarvon,AU,-24.8667,113.6333,77.07,73,0,16.11,clear sky
15,15,Labuhan,ID,-6.8844,112.2051,76.71,89,100,9.84,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
615,615,Djibo,BF,14.1022,-1.6306,84.74,9,0,9.93,clear sky
616,616,Thinadhoo,MV,0.5333,72.9333,82.45,73,60,12.35,broken clouds
617,617,Montevideo,UY,-34.8335,-56.1674,77.02,69,40,10.36,scattered clouds
619,619,Butterworth,MY,5.3991,100.3638,80.64,96,20,3.44,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

Null values present in the Preferred cities list


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Labuhan,ID,76.71,overcast clouds,-6.8844,112.2051,
6,Mar Del Plata,AR,78.82,clear sky,-38.0023,-57.5575,
7,Kapaa,US,70.20,light rain,22.0752,-159.3190,
8,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
15,Labuhan,ID,76.71,overcast clouds,-6.8844,112.2051,
16,Mar Del Plata,AR,78.82,clear sky,-38.0023,-57.5575,
17,Kapaa,US,70.20,light rain,22.0752,-159.3190,
18,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
25,Labuhan,ID,76.71,overcast clouds,-6.8844,112.2051,
26,Mar Del Plata,AR,78.82,clear sky,-38.0023,-57.5575,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
          hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print(f"Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Labuhan,ID,76.71,overcast clouds,-6.8844,112.2051,DoubleFAG
6,Mar Del Plata,AR,78.82,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
7,Kapaa,US,70.20,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,Hospitality Carnarvon
15,Labuhan,ID,76.71,overcast clouds,-6.8844,112.2051,DoubleFAG
...,...,...,...,...,...,...,...
615,Djibo,BF,84.74,clear sky,14.1022,-1.6306,Centre Des Ressources DPENA
616,Thinadhoo,MV,82.45,broken clouds,0.5333,72.9333,
617,Montevideo,UY,77.02,scattered clouds,-34.8335,-56.1674,KOLPING Hotel
619,Butterworth,MY,80.64,few clouds,5.3991,100.3638,Cititel Penang


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [26]:
# ********* Create a Customer Travel Destinations Map complete ***********